# DirTreeHtml (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

In [ ]:
#r @"../../../../../../../.nuget/packages/fsharp.control.asyncseq/3.2.1/lib/netstandard2.1/FSharp.Control.AsyncSeq.dll"
#r @"../../../../../../../.nuget/packages/system.reactive/6.0.1-preview.1/lib/net6.0/System.Reactive.dll"
#r @"../../../../../../../.nuget/packages/system.reactive.linq/6.0.1-preview.1/lib/netstandard2.0/System.Reactive.Linq.dll"
#r @"../../../../../../../.nuget/packages/argu/6.1.1/lib/netstandard2.0/Argu.dll"
#r @"../../../../../../../.nuget/packages/system.commandline/2.0.0-beta4.22272.1/lib/net6.0/System.CommandLine.dll"

In [ ]:
#!import ../nbs/Common.fs
#!import ../nbs/CommonFSharp.fs
#!import ../nbs/Async.fs
#!import ../nbs/AsyncSeq.fs
#!import ../nbs/Networking.fs
#!import ../nbs/Runtime.fs
#!import ../nbs/FileSystem.fs

In [ ]:
type FileSystemNode =
    | File of string * string
    | Folder of string * string * FileSystemNode list
    | Root of FileSystemNode list

let rec scanDirectory isRoot (basePath : string) (path : string) =
    let relativePath = path.Replace(basePath, "").Replace("\\", "/").TrimStart '/'
    let folderName = System.IO.Path.GetFileName path
    let subDirectories = System.IO.Directory.GetDirectories path |> Array.toList
    let files = System.IO.Directory.GetFiles path |> Array.toList

    let subDirectoryNodes = subDirectories |> List.map (scanDirectory false basePath)
    let fileNodes = files |> List.map (fun f -> File (System.IO.Path.GetFileName f, relativePath))

    let children = subDirectoryNodes @ fileNodes
    if isRoot
    then Root children
    else Folder (folderName, relativePath, children)

let rec generateHtml fsNode =
    match fsNode with
    | File (fileName, relativePath) ->
        $"""<li><span>&#128196; <a href="{relativePath}{if relativePath.Length = 0 then "" else "/"}{fileName}">{fileName}</a></span></li>"""
    | Folder (folderName, relativePath, children) ->
        let childrenHtml = children |> List.map generateHtml |> String.concat "\n"
        $"""<li><span>📁 <a href="{relativePath}">{folderName}</a></span><ul>{childrenHtml}</ul></li>"""
    | Root children ->
        children |> List.map generateHtml |> String.concat "\n"

let generateHtmlForFileSystem root =
    let rootHtml = root |> generateHtml
    $"""<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
</head>
<body>
  <ul>{rootHtml}</ul>
</body>
</html>
"""

In [ ]:
//// test

let path = "../../.github"
let html =
    scanDirectory true path path
    |> generateHtmlForFileSystem

html
|> _equal """<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
</head>
<body>
  <ul><li><span>📁 <a href="workflows">workflows</a></span><ul><li><span>&#128196; <a href="workflows/gh-pages.yaml">gh-pages.yaml</a></span></li></ul></li></ul>
</body>
</html>
"""

html |> Microsoft.DotNet.Interactive.Formatting.Html.ToHtmlContent

<!DOCTYPE html>
 
 
 
 
 
 📁 workflows 📄 gh-pages.yaml

<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
</head>
<body>
  <ul><li><span>📁 <a href="workflows">workflows</a></span><ul><li><span>&#128196; <a href="workflows/gh-pages.yaml">gh-pages.yaml</a></span></li></ul></li></ul>
</body>
</html>



## Arguments

In [ ]:
[<RequireQualifiedAccess>]
type Arguments =
    | [<Argu.ArguAttributes.ExactlyOnce>] Dir of string
    | [<Argu.ArguAttributes.ExactlyOnce>] Html of string

    interface Argu.IArgParserTemplate with
        member s.Usage =
            match s with
            | Dir _ -> nameof Arguments.Dir
            | Html _ -> nameof Arguments.Html

## main

In [ ]:
let main args =
    let argsMap = args |> Runtime.parseArgsMap<Arguments>

    let dir =
        match argsMap.[nameof Arguments.Dir] with
        | [ Arguments.Dir dir ] -> Some dir
        | _ -> None
        |> Option.get

    let htmlPath =
        match argsMap.[nameof Arguments.Html] with
        | [ Arguments.Html html ] -> Some html
        | _ -> None
        |> Option.get

    let fileSystem = scanDirectory true dir dir
    let html = generateHtmlForFileSystem fileSystem

    html |> FileSystem.writeAllTextAsync htmlPath
    |> Async.runWithTimeout 30000
    |> function
        | Some () -> 0
        | None -> 1

In [ ]:
//// test

let args =
    System.Environment.GetEnvironmentVariable "ARGS"
    |> Runtime.splitArgs
    |> Seq.toArray

match args with
| [||] -> 0
| args -> if main args = 0 then 0 else failwith "main failed"

0